In [ ]:
#install what needs to be installed setting up instance here
!pip uninstall -y bitsandbytes accelerate

%pip install -U bitsandbytes accelerate
%pip install langdetect

import bitsandbytes as bnb
import accelerate
from langdetect import detect

#Google Drive mount here
from google.colab import drive
drive.mount('/content/gdrive/')

In [ ]:
#enabling import of custom module for colab
! cp /content/gdrive/MyDrive/Util.py . 

In [ ]:
#set up file location here
import sys
sys.path.append('/content/gdrive/MyDrive')

# import libraries
from Util import load_data_JSON, load_model, write_out
import pandas as pd
from huggingface_hub import login
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, pipeline
import torch

In [ ]:
#load the data
general_df, prompt_g, correct_g, gtp4_g, Info_g = load_data_JSON('/content/gdrive/MyDrive/general_data.json', 'general')
qa_df, prompt_q, correct_q, gtp4_q, Info_q = load_data_JSON('/content/gdrive/MyDrive/qa_data.json', 'qa')
sum_df, prompt_s, correct_s, gtp4_s, Info_s = load_data_JSON('/content/gdrive/MyDrive/summarization_data.json', 'sum')

In [ ]:
# Access input (API key, etc.)
access = input('Access code?')

In [ ]:
#setup of model
tokenizer, model = load_model("meta-llama/Llama-2-7b-chat-hf", access)

In [ ]:
gen = pipeline("text-generation", model=model,torch_dtype = torch.float16, tokenizer=tokenizer)

In [ ]:
# System prompt to enforce English-only responses
system_prompt = """[INST] <>
You are a helpful assistant. You will answer only in English and avoid using any other languages. Your answers are clear and concise.
<>
"""

In [ ]:
# Language detection function to ensure English-only responses
def get_response(prompt, max_len):
    prompt_with_system = system_prompt + prompt + " [/INST]"
    retry_count = 0
    max_retries = 3
    response = ""

    while retry_count < max_retries:
        sequences = gen(prompt_with_system, do_sample=True, top_k=5, num_return_sequences=1, 
                        eos_token_id=tokenizer.eos_token_id, max_length=max_len + len(prompt))
        response = sequences[0]['generated_text']

        # Detect language of the response
        if detect(response) == 'en':
            return response
        else:
            retry_count += 1
            print(f"Detected non-English response, retrying ({retry_count}/{max_retries})...")

    # If it fails after retries, return the last attempt
    return response

In [ ]:
# ChainPoll method with language detection and retry mechanism
def chainpoll(model, tokenizer, prompts, num_responses=5):
    out_q = []
    max_retries = 3  # Limit retries for language detection

    for i, prompt in enumerate(prompts[:2]):
        hallucination_scores = []
        explanations = []

        prompt_with_system = system_prompt + f"{prompt} [/INST]"

        for _ in range(num_responses):
            retry_count = 0
            response = ""
            
            # Retry mechanism for ensuring English language responses
            while retry_count < max_retries:
                inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
                with torch.no_grad():
                    outputs = model.generate(
                        inputs['input_ids'],
                        max_length=50,
                        num_beams=5,
                        early_stopping=True
                    )
                response = tokenizer.decode(outputs[0], skip_special_tokens=True)
                
                if detect(response) == 'en':
                    break
                else:
                    retry_count += 1
                    print(f"Detected non-English response, retrying ({retry_count}/{max_retries})...")

            hallucination_prompt = f"Does the following output contain hallucinations? Explain in detail:\n\nOutput: {response}\n"
            hallucination_inputs = tokenizer(hallucination_prompt, return_tensors="pt").to("cuda")
            with torch.no_grad():
                hall_outputs = model.generate(
                    hallucination_inputs['input_ids'],
                    max_length=100,
                    num_beams=5,
                    early_stopping=True
                )

            hallucination_response = tokenizer.decode(hall_outputs[0], skip_special_tokens=True)

            if "yes" in hallucination_response.lower():
                hallucination_scores.append(1)
            else:
                hallucination_scores.append(0)

        hallucination_score = sum(hallucination_scores) / num_responses
        out_q.append({
            "prompt": prompt,
            "response": response,
            "hallucination_score": hallucination_score,
            "explanation": hallucination_response
        })

    return out_q

In [ ]:
chainpoll_results = chainpoll(model, tokenizer, prompt_q)

In [ ]:
# Format the output for better readability
formatted_out_q = []
for result in chainpoll_results:
    formatted_output = f"""
    Prompt: {result['prompt']}

    Response: {result['response']}

    Hallucination Score: {result['hallucination_score']}

    Explanation: {result['explanation']}
    """
    formatted_out_q.append(formatted_output)

# Print the formatted results (optional)
# for formatted_result in formatted_out_q:
    # print(formatted_result)

In [ ]:
out_q =[]
for result in chainpoll_results:
    prompt = result["prompt"]
    response = result["response"]
    hallucination_score = result["hallucination_score"]
    explanation = result["explanation"]

    # Append results to out_q for writing
    out_q.append({
        "prompt": prompt,
        "response": response,
        "hallucination_score": hallucination_score,
        "explanation": explanation
    })

In [ ]:
# Write the results
write_out("/content/gdrive/MyDrive/ChainPoll_qa_results.csv", formatted_out_q)